In [8]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import json
import altair as alt
import plotly.express as px
from data_processing import get_financial_quarter, get_financial_year, get_financial_year_quarter

In [9]:
pd.options.display.max_columns=50

In [10]:
# Function to load the configuration
def load_config():
    with open('config.json') as config_file:
        data = json.load(config_file)
    return data

# Load the configuration
config = load_config()

customer_data_filepath = config["file_paths"]["customer_file"]

In [11]:
df_customer = pd.read_excel(customer_data_filepath)

In [12]:
df_customer.head()

,customer_id,status,title,first_name,middle_name,last_name,date_of_birth,email,mobile,join_date,brand_name,agent,agent_type,heard_type,marketing,last_note,last_note_type,suburb,postcode,created_by
0,700000,Closed,NaN,Customer 1,Customer 2,Customer 3,2000-09-08,customer1@mate.com.au,123456789,2023-06-08,mate,7.000000e+09,Partner 1,Finder,False,2023-06-17 18:30:08.500428+10,Account information,jindalee,4074,70138
1,700001,Closed,NaN,Customer 2,Customer 3,Customer 4,1968-08-04,customer2@mate.com.au,123456789,2020-12-17,mate,7.000000e+09,Partner 2,Other,False,2023-07-08 08:34:03.347649+10,Account information,TOORMINA,2452,1004
2,700002,Closed,NaN,Customer 3,Customer 4,Customer 5,1974-08-03,customer1@mate.com.au,123456789,2023-06-07,mate,7.000000e+09,Partner 3,Google Search,False,2023-06-08 15:00:24.112279+10,Account information,REEDY CREEK,4227,70138
3,700003,Normal,NaN,Customer 4,Customer 5,Customer 6,1986-09-27,customer2@mate.com.au,123456789,2020-06-06,mate,NaN,NaN,Other,True,2023-10-06 00:06:47.438977+11,Account information,BEXLEY,2207,1004
4,700004,Closed,NaN,Customer 5,Customer 6,Customer 7,1985-02-02,customer1@mate.com.au,123456789,2023-04-06,mate,NaN,NaN,Other,True,2023-07-07 23:30:34.984002+10,Account information,Mernda,3754,70138


In [13]:
df_customer['join_year'] = df_customer['join_date'].dt.year
df_customer['join_month'] = df_customer['join_date'].dt.month
df_customer['join_quarter'] = df_customer['join_month'].apply(get_financial_quarter)
df_customer['join_fin_yr'] = df_customer['join_date'].apply(get_financial_year)

In [14]:
df_customer['join_fin_qtr'] = df_customer[['join_fin_yr','join_quarter']].apply(get_financial_year_quarter, axis=1)

In [16]:
# Extracting year and month from the 'join_date' column
customer_join_year = sorted(df_customer['join_year'].unique().tolist())
customer_join_fin_year = sorted(df_customer['join_fin_yr'].unique().tolist())
customer_join_fin_qtr = sorted(df_customer['join_fin_qtr'].unique().tolist())
#customer_join_month = df_customer['join_date'].dt.month.unique().tolist().sort()

# Displaying the result
customer_join_year


[2017, 2018, 2020, 2021, 2022, 2023]

In [18]:
customer_join_fin_qtr

['FY2017Q2',
 'FY2017Q3',
 'FY2017Q4',
 'FY2018Q1',
 'FY2019Q3',
 'FY2019Q4',
 'FY2020Q1',
 'FY2020Q2',
 'FY2020Q3',
 'FY2020Q4',
 'FY2021Q1',
 'FY2021Q4',
 'FY2022Q1',
 'FY2022Q3',
 'FY2022Q4',
 'FY2023Q1',
 'FY2023Q2']

In [ ]:


# Parameters for simulation
num_suppliers = 5
suppliers = [f"Supplier_{i}" for i in range(1, num_suppliers + 1)]
num_months = 24
new_orders_per_month = 5000
churned_orders_per_month = 2000
multi_subscription_ratio = 0.05


df_order = pd.read_excel("data/sampledata/Example report-order_export_example.xlsx")

# List of unique descriptions from the provided table
descriptions = df_order['description'].unique()

# Function to generate random datetime within a month
def random_date(year, month):
    start_date = datetime(year, month, 1)
    end_date = start_date + timedelta(days=28) # 28 days to ensure it's within the month
    return start_date + (end_date - start_date) * random.random()



In [ ]:
# Adjusting the simulation to include randomness and seasonality in the number of new orders per month

# Reset the lists for the new simulation
order_ids = []
customer_ids = []
subscription_ids = []
order_suppliers = []
statuses = []
order_types = []
descriptions_list = []
date_requested_list = []
date_created_list = []
date_modified_list = []

# Redefine order_id and subscription_id for the new simulation
order_id = 700000
subscription_id = 123461
customer_id = 600013

new_orders_per_month = 5000

# Function to simulate seasonal variation in new orders
def simulate_seasonal_orders(month):
    base_orders = new_orders_per_month
    if month in [11, 12, 1]:  # Higher orders in Nov, Dec, Jan
        base_orders *= 1.1  # 10% increase
    elif month in [6, 7, 8]:  # Lower orders in Jun, Jul, Aug
        base_orders *= 0.9  # 10% decrease
    return int(base_orders + random.randint(-500, 500))  # Random variation


# Simulate only new orders with seasonality
for month_offset in range(num_months):
    year = 2022 + month_offset // 12
    month = month_offset % 12 + 1

    num_new_orders = simulate_seasonal_orders(month)
    for _ in range(num_new_orders):
        order_ids.append(order_id)
        order_id += 1

        if random.random() < multi_subscription_ratio:
            customer_ids.append(random.randint(600008, customer_id - 1))
        else:
            customer_ids.append(customer_id)
            customer_id += 1

        subscription_ids.append(subscription_id)
        subscription_id += 1

        order_suppliers.append(random.choice(suppliers))
        statuses.append('Complete')
        order_types.append('New')
        descriptions_list.append(random.choice(descriptions))

        date_requested = random_date(year, month)
        date_requested_list.append(date_requested)
        date_created_list.append(date_requested)  # Assuming created and requested at the same time
        date_modified_list.append(date_requested + timedelta(days=random.randint(0, 5)))  # Modified within 5 days

# Create a DataFrame from the new simulated data
df_simulated_new_orders_seasonal = pd.DataFrame({
    'order_id': order_ids,
    'customer_id': customer_ids,
    'subscription_id': subscription_ids,
    'supplier': order_suppliers,
    'status': statuses,
    'order_type': order_types,
    'description': descriptions_list,
    'date_requested': date_requested_list,
    'date_created': date_created_list,
    'date_modified': date_modified_list
})

df_simulated_new_orders_seasonal.head()
